In [1]:
import requests
!pip install pygobject
!pip install gTTS
!pip install SpeechRecognition
!pip install playsound
!apt-get install -y portaudio19-dev
!pip install pyaudio
from gtts import gTTS
from IPython.display import Audio, display, Javascript
from google.colab import output
import pyaudio
import speech_recognition as sr
import os
import sys
import time
from datetime import datetime
import playsound




  Using cached pycairo-1.27.0.tar.gz (661 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
portaudio19-dev is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [2]:
#speak converted audio to text
def speak(text):
    tts = gTTS(text=text, lang='pt')
    filename = "voice.mp3"
    try:
        os.remove(filename)
    except OSError:
        pass
    tts.save(filename)
    #playsound.playsound(filename)
    # Reproduz o áudio no Colab
    display(Audio(filename, autoplay=True))
    time.sleep(5)  # Ajuste o tempo com base na duração do seu áudio

In [3]:
#function to respond to commands
def respond(text):
    print("Texto extraido " + text)
    if 'previsão do tempo' in text:
        speak(weather_today())
    elif 'dólar' in text:
        speak(dolar())
    elif 'euro' in text:
        speak(euro())
    elif 'horário' in text:
        strTime = datetime.today().strftime("%H:%M %p")
        print(strTime)
        speak(strTime)
    elif 'sair' in text:
        speak("Até a próxima!")
        #exit()
        sys.exit()
    else:
        # Resposta padrão quando o texto não corresponder a nada
        speak("Desculpe, não entendi. Pode tentar novamente?")

In [4]:
# Dicionário com os códigos de tempo e suas descrições
weather_codes = {
    "0": "Céu aberto",
    "1": "Ceu aberto",
    "2": "parcialmente nublado",
    "3": "nublado",
    "45": "Nevoeiro",
    "48": "depositando névoa de geada",
    "51": "Chuva: Leve",
    "53": "Chuva moderada",
    "55": "Chuva densa de intensidade",
    "56": "Chuva congelante: Fraca",
    "57": "Chuva Congelante: forte intensidade",
    "61": "Chuva: Fraca",
    "63": "Chuva: moderada",
    "65": "Chuva: forte intensidade",
    "66": "Chuva Congelante: Fraca",
    "67": "Chuva Congelante: forte intensidade",
    "71": "Queda de Neve: Leve",
    "73": "Queda de Neve: Moderada",
    "75": "Queda de neve: forte intensidade",
    "77": "Grãos de neve",
    "80": "Chuva fraca de curta duração",
    "81": "Chuva moderada de curta duração",
    "82": "Chuva forte de curta duração",
    "85": "Aguaceiros de neve ligeiros",
    "86": "Aguaceiros de neve fortes",
    "95": "Tempestade: Fraca ou moderada",
    "96": "Tempestade com granizo leve",
    "99": "Trovoada com granizo forte"
}


In [5]:
def get_weather_data(latitude, longitude):
    url = f'https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&timezone=America/Sao_Paulo&current_weather=true&daily=temperature_2m_max,temperature_2m_min,windspeed_10m_max,rain_sum'

    try:
        response = requests.get(url, timeout=(30, 40), headers={'Content-Type': 'application/json'})
        response.raise_for_status()  # Verifica se houve erro na requisição (status code 4xx ou 5xx)
        return response.json()  # Retorna a resposta JSON como um dicionário
    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição: {e}")
        return None

In [6]:
# Função para obter a descrição do código do tempo
def get_weather_description(weather_code):
    return weather_codes.get(str(weather_code), "Código desconhecido")

In [7]:
def weather_today():
  latitude = -23.5505
  longitude = -46.6333
  data = get_weather_data(latitude, longitude)
  # Acessando os dados dentro de 'current_weather'
  current_weather = data['current_weather']
  temperature = current_weather['temperature']
  windspeed = current_weather['windspeed']
  weathercode = current_weather['weathercode']
  description = get_weather_description(weathercode)
  # Exibindo os resultados
  print(f"Temperature: {temperature} °C")
  print(f"Windspeed: {windspeed} km/h")
  print(f"Weather code: {weathercode}")
  print(f"Description: {description}")
  return f'Hoje o clima está: {description} e a temperatura é de {temperature} graus'




In [8]:
#codigo não utilizado
text_to_say = "Um palíndromo é uma palavra ou frase que pode ser lida no seu sentido normal, da esquerda para a direita, bem como no sentido contrário, da direita para a esquerda, sem que haja mudança nas palavras que a formam e no seu significado."
language = "pt"
gtts_object = gTTS(text = text_to_say,
                  lang = language,
                  slow = False)
gtts_object.save("/content/gtts.wav")
Audio("/content/gtts.wav")

In [9]:
def euro():
    url = 'https://api.exchangerate-api.com/v4/latest/EUR'  # A URL da API
    try:
        response = requests.get(url)
        response.raise_for_status()  # Vai gerar um erro se o status code não for 2xx

        data = response.json()  # Converte a resposta para JSON
        taxa_brl = data['rates']['BRL']  # Acessa a taxa de EUR para BRL
        return f'A taxa de EURO para REAL é: {taxa_brl}'

    except requests.exceptions.HTTPError as http_err:
        #return f'Erro HTTP: {http_err}'  # Tratamento de erro para falhas HTTP
        return 'Não foi possível obter a cotação desta moeda'
    except Exception as err:
        #return f'Erro ao acessar a API: {err}'  # Captura outros erros
        return 'Não foi possível obter a cotação desta moeda'

In [10]:
def dolar():
    url = 'https://api.exchangerate-api.com/v4/latest/USD'  # A URL da API
    try:
        response = requests.get(url)
        response.raise_for_status()  # Vai gerar um erro se o status code não for 2xx

        data = response.json()  # Converte a resposta para JSON
        taxa_brl = data['rates']['BRL']  # Acessa a taxa de EUR para BRL
        return f'A taxa de DOLAR para REAL é: {taxa_brl}'

    except requests.exceptions.HTTPError as http_err:
        #return f'Erro HTTP: {http_err}'  # Tratamento de erro para falhas HTTP
        return 'Não foi possível obter a cotação desta moeda'
    except Exception as err:
        #return f'Erro ao acessar a API: {err}'  # Captura outros erros
        return 'Não foi possível obter a cotação desta moeda'

In [11]:
#Teste do assistente
#while True:
#text = "Qual a previsão do tempo para hoje?"
#text = "Quanto vale um euro?"
#text = "Quanto vale um dólar?"
#text = "Qual o horário atual?"
#respond(text)
while True:
    text = input("Digite algo (ou 'sair' para encerrar): ")
    respond(text)

Digite algo (ou 'sair' para encerrar): qual o horário atual?
Texto extraido qual o horário atual?
19:10 PM


Digite algo (ou 'sair' para encerrar): sair
Texto extraido sair


SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
